In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
pd.options.display.max_columns = 100

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.genome import get_feature_hit_set, is_overlap
from util.operon import get_operon_name_set, get_operon_ID_set
from util.feature import get_feat_d_from_ID
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD

In [2]:
exp_muts_df = pd.read_pickle("./data/7_df.pkl")

display(len(exp_muts_df), exp_muts_df.head())

114

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
2527,28,None,[crl],DEL,257908,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(257908, 258683)","{ECK120001080, ECK125229291, ECK125229290}","[{'name': 'insA9', 'RegulonDB ID': 'ECK1252292...",False,False,{},{},{},{},{},True,"[{'name': 'insB9', 'RegulonDB ID': 'ECK1252292...","{'ECK125229291': ['ECK125229291'], 'ECK1252292...",[],{},[],{},[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2528,3,coding (163/210 nt),cspC,INS,1907273,NC_000913,(CGTCCTG)1→2,1,TOL_putrescine,50,1,1.0,1,True,"(1907273, 1907273)",{ECK120002092},"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",False,False,{},{},{},{},{},True,"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",{'ECK120002092': ['ECK120002092']},"[{'name': 'yobF-cspC', 'RegulonDB ID': 'ECK120...",{'ECK120028784': ['ECK120002092']},[],{},[],{},[{'name': 'Transcription'}],{'Transcription': ['ECK120028784']},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2529,29,None,"insB1, insA",DEL,1978503,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(1978503, 1979278)","{ECK120023915, ECK120003617, ECK120003616}","[{'name': 'insB-5', 'RegulonDB ID': 'ECK120003...",False,False,"{ECK120017150, ECK120013472}","{ECK125095446, ECK120034252}",{},{},{},True,"[{'name': 'flhDC TFBS', 'RegulonDB ID': 'ECK12...","{'ECK120003616': ['ECK120013472', 'ECK12000361...","[{'name': 'flhDC', 'RegulonDB ID': 'ECK1200149...","{'ECK120014934': ['ECK120003616', 'ECK12000361...","[{'name': 'YjjQ', 'significantly associated co...","{'YjjQ': ['flhDC', 'flhDC'], 'Fur': ['flhDC', ...",[],{},"[{'name': 'Mobilome: prophages, transposons'},...","{'Mobilome: prophages, transposons': ['ECK1250...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2530,32,intergenic (‑1/+1),gatC/gatC,DEL,2173363,NC_000913,Δ2 bp,1,TOL_putrescine,50,1,1.0,1,False,"(2173363, 2173364)",{ECK120002276},"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",False,False,{},{},{},{},{},False,"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",{'ECK120002276': ['ECK120002276']},"[{'name': 'gatYZABCD', 'RegulonDB ID': 'ECK120...",{'ECK120015849': ['ECK120002276']},"[{'name': 'GatR', 'significantly associated co...","{'GatR': ['gatYZABCD'], 'ArcA': ['gatYZABCD'],...",[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2531,4,N34K (AAT→AAG),mreB,SNP,3400986,NC_000913,A→C,1,TOL_putrescine,50,1,1.0,1,True,"(3400986, 3400986)",{ECK120000601},"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",False,False,{},{},{},{},{},True,"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",{'ECK120000601': ['ECK120000601']},"[{'name': 'mreBCD', 'RegulonDB ID': 'ECK120030...",{'ECK120030764': ['ECK120000601']},[],{},[],{},"[{'name': 'Cell cycle control, cell division, ...","{'Cell cycle control, cell division, chromosom...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


In [3]:
operon_df = pd.read_pickle("./data/operon_df.pkl")
operon_df.head()

,OPERON_ID,OPERON_NAME,FIRSTGENEPOSLEFT,LASTGENEPOSRIGHT,REGULATIONPOSLEFT,REGULATIONPOSRIGHT,OPERON_STRAND,OPERON_INTERNAL_COMMENT,KEY_ID_ORG,range
0,ECK120011191,rob,4634441,4635310,4634441,4635382,reverse,NaN,ECK12,"(4634441, 4635382)"
1,ECK120011203,argP,3059753,3060646,3059684,3060680,forward,NaN,ECK12,"(3059684, 3060680)"
2,ECK120011329,lrhA,2405703,2406641,2405703,2406866,reverse,NaN,ECK12,"(2405703, 2406866)"
3,ECK120011490,rtcR,3558268,3559866,3558268,3559866,forward,NaN,ECK12,"(3558268, 3559866)"
4,ECK120011629,lrp,932595,933089,932265,933089,forward,NaN,ECK12,"(932265, 933089)"


In [4]:
TU_df = pd.read_csv("./data/RegulonDB10/transcription_unit.txt", sep="\t", comment='#', header=None)
TU_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "PROMOTER_ID",
    "TRANSCRIPTION_UNIT_NAME",
    "OPERON_ID",
    "TRANSCRIPTION_UNIT_NOTE",
    "TU_INTERNAL_COMMENT",
    "KEY_ID_ORG"]


TU_objects_df = pd.read_csv("./data/RegulonDB10/tu_objects_tmp.txt",
                            sep="\t",
                            comment='#',
                            header=None,
                            )

TU_objects_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "NUMTU",
    "TU_POSLEFT",
    "TU_POSRIGHT",
    "TU_TYPE",
    "TU_OBJECT_CLASS",
    "TU_OBJECT_ID",
    "TU_OBJECT_NAME",
    "TU_OBJECT_POSLEFT",
    "TU_OBJECT_POSRIGHT",
    "TU_OBJECT_STRAND",
    "TU_OBJECT_COLORCLASS",
    "TU_OBJECT_DESCRIPTION",
    "TU_OBJECT_SIGMA",
    "TU_OBJECT_EVIDENCE",
    "TU_OBJECT_RI_TYPE",
    "TU_OBJECT_TYPE",
    "EVIDENCE"
]

TU_objects_df["TU range"] = TU_objects_df.apply(
    lambda row: (row["TU_POSLEFT"], row["TU_POSRIGHT"]), axis=1)

TU_objects_df.head()

,TRANSCRIPTION_UNIT_ID,NUMTU,TU_POSLEFT,TU_POSRIGHT,TU_TYPE,TU_OBJECT_CLASS,TU_OBJECT_ID,TU_OBJECT_NAME,TU_OBJECT_POSLEFT,TU_OBJECT_POSRIGHT,TU_OBJECT_STRAND,TU_OBJECT_COLORCLASS,TU_OBJECT_DESCRIPTION,TU_OBJECT_SIGMA,TU_OBJECT_EVIDENCE,TU_OBJECT_RI_TYPE,TU_OBJECT_TYPE,EVIDENCE,TU range
0,ECK120008913,3,1825955,1832013,H,PM,ECK120009851,astCp1,1832013,1832013,R,NaN,NaN,Sigma70,Human inference of promoter position,NaN,predicted,NaN,"(1825955, 1832013)"
1,ECK120008913,3,1825955,1832013,H,GN,ECK120003528,astE,1825955,1826923,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
2,ECK120008913,3,1825955,1832013,H,GN,ECK120003529,astB,1826916,1828259,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
3,ECK120008913,3,1825955,1832013,H,GN,ECK120003532,astC,1830762,1831982,R,"0,0,255",nitrogen metabolism,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"
4,ECK120008913,3,1825955,1832013,H,GN,ECK120003530,astD,1828256,1829734,R,"51,153,255",amino acids,NaN,NaN,NaN,predicted,NaN,"(1825955, 1832013)"


# significance

In [5]:
unknown_op_len_df = pd.read_pickle("./data/operon_length_df.pkl")
unknown_op_len_df.head()

,operon description,length
0,unknown,265905


In [6]:
from util.feature import get_feat_d

In [7]:
# a bit behind annotations operons, etc.
# Still uses same approach to counted, just not all data-types have been consolidated into the operon feat col

feat_col_name = "operons"
link_col_name = "operon links"
comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r[link_col_name].items():
        
        f_d = dict()
        if f == "unknown":
            f_d["RegulonDB ID"] = "unknown"
            f_d["name"] = "unknown"
            f_d["range"] = (0, unknown_op_len_df[unknown_op_len_df["operon description"]=="unknown"].iloc[0]["length"])
        else:
            f_d = get_feat_d(RegulonDB_ID=f, json=r[feat_col_name])
        
        if f_d["RegulonDB ID"] not in comp_mut_df.index:
            f_len = f_d["range"][1] - f_d["range"][0] + 1
            df = pd.DataFrame({"name": f_d["name"], "length": f_len, "observed mutation count": len(links)},
                              index=[f_d["RegulonDB ID"]])  # "name" column just for visual inspection
            comp_mut_df = comp_mut_df.append(df, sort=False)
        else:
            comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count,name,length
ECK120028784,3,yobF-cspC,643.0
ECK120014934,21,flhDC,1312.0
ECK125095389,28,insAB-5A-5B-5,709.0
ECK120015849,7,gatYZABCD,5518.0
ECK120030764,5,mreBCD,2970.0
ECK120014590,7,glpEGR,2165.0
ECK120016992,6,rplKAJL-rpoBC,11376.0
ECK120014574,2,edd-eda,2749.0
ECK120014803,1,fliFGHIJK,6460.0
ECK120014418,1,iscRSUA,2702.0


In [8]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["operons"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["RegulonDB ID"]]["observed mutation count"]

In [9]:
mutated_seq_len = np.sum(comp_mut_df["length"])

# XXX Is this correct. What about overlapping operons?
# They would make the genome remainder smaller than what it should be.
TOTAL_GENOME_LEN = 4641652
genome_remainder_len = TOTAL_GENOME_LEN - mutated_seq_len

# Not account for the number of mutations that don't land in operons.
df = pd.DataFrame([[0, genome_remainder_len]], columns=["observed mutation count", "length"], index=["genome remainder"])
comp_mut_df = comp_mut_df.append(df, sort=False)
total_seq_len = TOTAL_GENOME_LEN


comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

,observed mutation count,name,length,proportion
ECK120028784,3,yobF-cspC,643.0,0.000139
ECK120014934,21,flhDC,1312.0,0.000283
ECK125095389,28,insAB-5A-5B-5,709.0,0.000153
ECK120015849,7,gatYZABCD,5518.0,0.001189
ECK120030764,5,mreBCD,2970.0,0.000640


In [10]:
from collections import Counter
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=num_muts,
        replace=True,
        p=proportion_l
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,name,length,proportion,equal or larger count,p value,significant
ECK120028784,3,yobF-cspC,643.0,0.000139,1,0.0001,True
ECK120014934,21,flhDC,1312.0,0.000283,0,0.0000,True
ECK125095389,28,insAB-5A-5B-5,709.0,0.000153,0,0.0000,True
ECK120015849,7,gatYZABCD,5518.0,0.001189,0,0.0000,True
ECK120030764,5,mreBCD,2970.0,0.000640,0,0.0000,True


In [11]:
from statsmodels.stats import multitest

pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

# pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,name,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
ECK120028784,3,yobF-cspC,643.0,0.000139,1,0.0001,True,0.0054,True
ECK120014934,21,flhDC,1312.0,0.000283,0,0.0000,True,0.0000,True
ECK125095389,28,insAB-5A-5B-5,709.0,0.000153,0,0.0000,True,0.0000,True
ECK120015849,7,gatYZABCD,5518.0,0.001189,0,0.0000,True,0.0000,True
ECK120030764,5,mreBCD,2970.0,0.000640,0,0.0000,True,0.0000,True


In [12]:
# not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
comp_mut_df.head()

,observed mutation count,name,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance,final significance
ECK120028784,3,yobF-cspC,643.0,0.000139,1,0.0001,True,0.0054,True,True
ECK120014934,21,flhDC,1312.0,0.000283,0,0.0000,True,0.0000,True,True
ECK125095389,28,insAB-5A-5B-5,709.0,0.000153,0,0.0000,True,0.0000,True,True
ECK120015849,7,gatYZABCD,5518.0,0.001189,0,0.0000,True,0.0000,True,True
ECK120030764,5,mreBCD,2970.0,0.000640,0,0.0000,True,0.0000,True,True


In [13]:
for i, r in exp_muts_df.iterrows():
    for d in r["operons"]:
        if d["RegulonDB ID"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["RegulonDB ID"]]["final significance"]
#             d["significant"] = comp_mut_df.loc[d["RegulonDB ID"]]["final significance"]

# Outputting dataframe for the next step in the pipeline

In [14]:
assert(len(exp_muts_df.exp.unique())==1)
exp = exp_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_op_signif.csv")

In [15]:
exp_muts_df.to_pickle("./data/8_df.pkl")

In [16]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_ID(RegulonDB_ID, json):
#     name = ""
#     is_signif = False
#     for d in json:
#         if d["RegulonDB ID"] == RegulonDB_ID:
#             name = d["name"]
#             is_signif = d["significant"]
#             break
#     return (name, is_signif)


# targ_cnt_d = dict()w
# for _, r in exp_muts_df.iterrows():
#     link_d = r["operon links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["operons"])
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0", width=1)
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.1, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 operon mutated feature count")
# ax.get_legend().remove()

In [17]:
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["operon links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["operons"])
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,13),
#                   color="#4C72B0", width=1)
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.35, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Significant operons")
# ax.get_legend().remove()